# Aula 10 - Thrust para programação paralela em GPU

In [1]:
!nvidia-smi

Fri Sep 13 13:17:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Exemplo 1

In [2]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Writing exemplo1.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [4]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


## Exemplo 2

In [5]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [7]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


## Stocks Google

In [14]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Overwriting stocks.cu


In [15]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [16]:
  !./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.000593141
Tempo de CÓPIA (em segundos)  0.217349


# Exercicios:

## Exercicio 1

In [18]:
%%writefile ex1.cu
#include <thrust/device_vector.h>   // Biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca para entrada e saída de dados
#include <fstream>                  // Biblioteca para leitura de arquivos
#include <chrono>                   // Biblioteca para medir o tempo de execução

int main() {
    // Abrir o arquivo que contém os preços das ações
    std::ifstream file("stocks-google.txt");

    // Verificar se o arquivo foi aberto corretamente
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return -1;
    }

    // Vetor para armazenar os valores lidos do arquivo
    thrust::host_vector<double> host;

    // Variável auxiliar para armazenar cada valor lido do arquivo
    double value;

    // Início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Ler todos os valores do arquivo e armazená-los no vetor host
    while (file >> value) {
        host.push_back(value);
    }

    // Fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Fechar o arquivo após a leitura
    file.close();

    // Calcular o tempo de leitura
    std::chrono::duration<double> leitura_diff = leitura_f - leitura_i;
    std::cout << "Tempo de LEITURA (em segundos): " << leitura_diff.count() << std::endl;

    // Início da medição do tempo de cópia dos dados para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Transferir os dados do host_vector (CPU) para o device_vector (GPU)
    thrust::device_vector<double> dev(host);

    // Fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcular o tempo de cópia
    std::chrono::duration<double> copia_diff = copia_f - copia_i;
    std::cout << "Tempo de CÓPIA (em segundos): " << copia_diff.count() << std::endl;

    return 0;
}


Writing ex1.cu


In [19]:
!nvcc -arch=sm_75 -std=c++14 ex1.cu -o ex1

In [20]:
!./ex1 < stocks-google.txt

Tempo de LEITURA (em segundos): 0.000500838
Tempo de CÓPIA (em segundos): 0.196905


## Exercicio 2

In [21]:
%%writefile ex2.cu
#include <thrust/device_vector.h>   // Biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Biblioteca Thrust para vetores na CPU
#include <thrust/reduce.h>          // Biblioteca Thrust para operações de redução
#include <thrust/extrema.h>         // Biblioteca Thrust para encontrar máximo e mínimo
#include <iostream>                 // Biblioteca para entrada e saída de dados
#include <fstream>                  // Biblioteca para leitura de arquivos
#include <chrono>                   // Biblioteca para medir o tempo de execução

int main() {
    // Abrir o arquivo que contém os preços das ações
    std::ifstream file("stocks-google.txt");

    // Verificar se o arquivo foi aberto corretamente
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return -1;
    }

    // Vetor para armazenar os valores lidos do arquivo
    thrust::host_vector<double> host;

    // Variável auxiliar para armazenar cada valor lido do arquivo
    double value;

    // Ler todos os valores do arquivo e armazená-los no vetor host
    while (file >> value) {
        host.push_back(value);
    }

    // Fechar o arquivo após a leitura
    file.close();

    // Transferir os dados do host_vector (CPU) para o device_vector (GPU)
    thrust::device_vector<double> dev(host);

    // --- Cálculo de preços médios e extremos ---

    int total_dias = dev.size();
    int ultimo_ano_dias = 365;  // Usaremos os últimos 365 dias

    // 1. Preço médio total (soma de todos os elementos / número de elementos)
    double soma_total = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double media_total = soma_total / total_dias;
    std::cout << "Preço médio total dos últimos 10 anos: " << media_total << std::endl;

    // 2. Preço médio dos últimos 365 dias
    if (total_dias >= 365) {
        double soma_ultimo_ano = thrust::reduce(dev.end() - ultimo_ano_dias, dev.end(), 0.0, thrust::plus<double>());
        double media_ultimo_ano = soma_ultimo_ano / ultimo_ano_dias;
        std::cout << "Preço médio dos últimos 365 dias: " << media_ultimo_ano << std::endl;
    } else {
        std::cout << "Não há dados suficientes para calcular o preço médio dos últimos 365 dias." << std::endl;
    }

    // 3. Maior e menor preço total
    auto max_total = thrust::max_element(dev.begin(), dev.end());
    auto min_total = thrust::min_element(dev.begin(), dev.end());
    std::cout << "Maior preço total dos últimos 10 anos: " << *max_total << std::endl;
    std::cout << "Menor preço total dos últimos 10 anos: " << *min_total << std::endl;

    // 4. Maior e menor preço dos últimos 365 dias
    if (total_dias >= 365) {
        auto max_ultimo_ano = thrust::max_element(dev.end() - ultimo_ano_dias, dev.end());
        auto min_ultimo_ano = thrust::min_element(dev.end() - ultimo_ano_dias, dev.end());
        std::cout << "Maior preço dos últimos 365 dias: " << *max_ultimo_ano << std::endl;
        std::cout << "Menor preço dos últimos 365 dias: " << *min_ultimo_ano << std::endl;
    } else {
        std::cout << "Não há dados suficientes para calcular os extremos dos últimos 365 dias." << std::endl;
    }

    return 0;
}


Writing ex2.cu


In [22]:
!nvcc -arch=sm_75 -std=c++14 ex2.cu -o ex2

In [23]:
!./ex2 < stocks-google.txt

Preço médio total dos últimos 10 anos: 1580.08
Preço médio dos últimos 365 dias: 1615.81
Maior preço total dos últimos 10 anos: 2200
Menor preço total dos últimos 10 anos: 1015.24
Maior preço dos últimos 365 dias: 2200
Menor preço dos últimos 365 dias: 1060.5


## Exercicio 3

In [30]:
%%writefile ex3.cu
#include <thrust/device_vector.h>   // Biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Biblioteca Thrust para vetores na CPU
#include <thrust/transform.h>       // Biblioteca Thrust para transformar vetores
#include <thrust/reduce.h>          // Biblioteca Thrust para operações de redução
#include <thrust/functional.h>      // Operações como subtração
#include <iostream>                 // Biblioteca para entrada e saída de dados
#include <fstream>                  // Biblioteca para leitura de arquivos

int main() {
    // Abrir o arquivo que contém os preços das ações
    std::ifstream file("stocks2.txt");

    // Verificar se o arquivo foi aberto corretamente
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo!" << std::endl;
        return -1;
    }

    // Vetores para armazenar os preços da Apple e da Microsoft
    thrust::host_vector<double> AAPL_prices;
    thrust::host_vector<double> MSFT_prices;

    double aapl_price, msft_price;

    // Ler os dados do arquivo
    while (file >> aapl_price) {
        char comma; // Para ignorar a vírgula entre os valores
        file >> comma >> msft_price;
        AAPL_prices.push_back(aapl_price);
        MSFT_prices.push_back(msft_price);
    }

    // Fechar o arquivo
    file.close();

    // Transferir os dados para a GPU
    thrust::device_vector<double> AAPL_dev = AAPL_prices;
    thrust::device_vector<double> MSFT_dev = MSFT_prices;

    // Vetor para armazenar as diferenças entre os preços da Apple e Microsoft
    thrust::device_vector<double> diff(AAPL_dev.size());

    // Calcular a diferença ponto a ponto
    thrust::transform(AAPL_dev.begin(), AAPL_dev.end(), MSFT_dev.begin(), diff.begin(), thrust::minus<double>());

    // Calcular a soma das diferenças
    double soma_diferencas = thrust::reduce(diff.begin(), diff.end(), 0.0, thrust::plus<double>());

    // Calcular a média das diferenças
    double media_diferencas = soma_diferencas / diff.size();

    // Exibir a média das diferenças
    std::cout << "A diferença média entre os preços das ações da Apple e Microsoft é: " << media_diferencas << std::endl;

    return 0;
}


Overwriting ex3.cu


In [31]:
!nvcc -arch=sm_75 -std=c++14 ex3.cu -o ex3

In [32]:
!./ex3

A diferença média entre os preços das ações da Apple e Microsoft é: -265.232
